# Insert User data via GraphQL

## Goal

- Create user objects including 'name' and 'address' fields
  - 'name' field: 'user1', 'user2' ...
  - 'address' field: import from database 'Cities in Japan'
    [generated in another notebook](./scraping%20to%20GraphQL.ipynb)
- Generate GraphQL

In [1]:
import pandas as pd
LEN=1000
df_user = pd.DataFrame(index=list(range(LEN)))
df_user['name'] = df_user.index.astype(str).map(lambda x: 'user'+x)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_user[df_user.index < 3]

,name
0,user0
1,user1
2,user2


In [3]:
from graphqlclient import GraphQLClient
import json
import random
client = GraphQLClient('http://10.0.2.2:4000')


In [7]:
def getAddressList():
    result = client.execute('''
    {
      locations {
        address
      }
    }
    ''')
    result = json.loads(result)

    if not ('data' in result): 
        return None
    result_data = result['data']
    if not ('locations' in result_data): 
        return None
    return list(map(lambda x: x['address'], result_data['locations']))
list_addresses = getAddressList()
df_user['address'] = pd.Series( random.choices(list_addresses, k=len(df_user)), index=df_user.index )

In [8]:
df_user[df_user.index < 3]

,name,address
0,user0,Minamata
1,user1,Miyoshi
2,user2,Yamagata


In [13]:
def execMutationCreateUser(name, address):
    user = {
        'name': name,
        'address': address
    }
    result = client.execute('''
    mutation CreateUserMutation($name: String!, $address: String!) {
    createUser(data: {
        name: $name
        address: $address
      }) {
        name
        address
      }
    }
    ''', variables=json.dumps(user))

def iterateExecMutation(df):
    for _, row in df.iterrows():
        execMutationCreateUser(address=row['address'], name=row['name'])

In [15]:
iterateExecMutation(df_user)